Работа по анализу данных Lenta.ru

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import multiprocessing
import spacy  # For preprocessing
from gensim.models import Word2Vec, KeyedVectors
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from sklearn.manifold import TSNE
from numpy import dot
from numpy.linalg import norm

1-ый этап: 
* вывод нужных столбцов
* очистка от лишних символов, цифр, приведение к одному регистру 
* запись в новый файл очищенные данные

In [ ]:
#выделить из данных только необхдимые столбцы title, text, topic
old_questions = pd.read_csv("../input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv")
keep_col = ['title', 'text', 'topic']
questions = old_questions[keep_col]
questions.to_csv("newData.csv", index = False)

In [ ]:
questions.tail()

In [ ]:
questions.head()#вывод 5-ти первых строк

In [ ]:
#проверка выводы нового док-та, в котором только нужные столбцы
df = pd.read_csv("./newData.csv")
df.head()

In [ ]:
#вывод общих тем(планировалось использовать при кластеризации)
questions.groupby("topic").count()


In [ ]:
#кол-во топиков
print(len(questions.groupby("topic")))

In [ ]:

#очистка данных от цифр, знаков препинания, все под
#один регистр
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^А-Яа-я() A-Za-z()\'\-\`\"\_\n]", " ")
    #df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(str)
    return df
questions = df.dropna() #удаление строк с содержанием NaN

#еще и без запятых

questions = standardize_text(questions, "title")
questions.to_csv("clean_data.csv")





In [ ]:
questions = df.dropna() #удаление строк с содержанием NaN
questions = standardize_text(questions, "text")
questions.to_csv("clean_data.csv")

In [ ]:
questions.tail()

2-ой этап: 
* токенизация очищенного файла
* попытка убрать стоп слова
* лемматизация и стемминг(потом не используется в пользу лемматизатора) (узнать одинаково ли хорошо они представляют слова в данной выборке)

In [ ]:
import pandas as pd
df = pd.read_csv("./clean_data.csv")

In [ ]:
df["all"] = df["text"] + "" + df["title"]

In [ ]:
df["all"].head(7)

In [ ]:
df["all"].tail(50)

In [ ]:
#токенизация
import nltk
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('[А-я]\w+')

df["all"] = df["text"].apply(lambda x: tokenizer.tokenize(x.lower()))
df["all"].head(7)

In [ ]:
df["all"].to_csv("tokens.csv")

In [ ]:
import pandas as pd
df = pd.read_csv("../input/tokens/tokens (2).csv")

In [ ]:
df.head(7)

In [ ]:
df.tail(15)

In [ ]:
from pymystem3 import Mystem
m = Mystem()

In [ ]:

from pymystem3 import Mystem
m = Mystem()
m.lemmatize(df)[0]

пришлось сохранить токены в один файл, тк использовать будем их и для кеггл перегруз оперативки

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words("russian"))

In [ ]:
#слишком долго идет обработка от стопслов
#не очистилось, тк долго грузило блокнот
from nltk.corpus import stopwords

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('russian')]
    return words
df["big_model"] = df["big_model"].apply(lambda x: remove_stopwords(x))
df["big_model"].head(4)

In [ ]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian")

In [ ]:
first_1000 = df["all"][:1000]
for word in first_1000:
    print (stemmer.stem(word))

In [ ]:
!pip install pymystem3

In [ ]:
#лемматизация 
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

sent = df["all"].apply(lambda x: word_lemmatizer(df["all"]))
sent.head(4)

In [ ]:
#стемминг для сравнения, потом не используется в порзу лемматизатора
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text
df["big_model"] = df["big_model"].apply(lambda x: word_stemmer(x))
df["big_model"].head(4)

3-ий этап: 
* обучение модели word2vec и ее сохранение
* вывод словаря 
* вывод векторов слов
* сравнение слов на схожесть

In [ ]:
import pandas as pd
md = pd.read_csv("../input/lemmatize2/csvexample3.csv")
md.head(5)

In [ ]:
import nltk

import re

import numpy as np
word2count = {}

for data in md["бой"][:1000]:

    words = nltk.word_tokenize(data)

    for word in words:

        if word not in word2count.keys():

            word2count[word] = 1
            
        else:

            word2count[word] += 1

In [ ]:
print(word2count)

In [ ]:

import heapq

freq_words = heapq.nlargest(100, word2count, key=word2count.get)

In [ ]:
print(freq_words)

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(
    min_count=10,
    window=2,
    size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [ ]:
w2v_model.build_vocab(md["бой"])

In [ ]:
w2v_model.train(md["бой"], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.save("word2vec.w2v_model")

In [ ]:
len(w2v_model.wv.vocab)

In [ ]:
words = sorted(w2v_model.wv.vocab.keys())
print (words[30:50])

In [ ]:
print(w2v_model.wv.vocab)

In [ ]:
w2v_model.wv.most_similar(positive=["россия"])

In [ ]:
words = list(w2v_model.wv.vocab)
print(words)

In [ ]:
from gensim.models import Word2Vec
#ошибка (Your notebook tried to allocate more memory than is available. It has restarted.)
model = Word2Vec(md["бой"], min_count=15,size= 300,workers=8, window =5, sg = 1)

In [ ]:
model.save("word2vec.model")

In [ ]:
#вывод словаря слов модели
words = list(model.wv.vocab)
print(words)

In [ ]:

model["кореи"]

In [ ]:
model["лица"]

In [ ]:
model.similarity( "северная", "корея")
#сходство двух слов

евклидово подобие

In [ ]:
model.most_similar('успешно')[:5]
#5 самых популярных значений 

In [ ]:

model.most_similar('кореи')[:5]

In [ ]:
#это перевод словаря в матрицу

import pandas as pd


points = pd.DataFrame(model[words]).T

print(points)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from sklearn.cluster import KMeans


#from sklearn.datasets.samples_generator import make_blobs

kmeans = KMeans(n_clusters = len(questions.groupby("topic"))) #23

kmeans.fit(points)
y_kmeans = kmeans.predict(points)
plt.scatter(points.iloc[:, 0], points.iloc[:, 1], c = y_kmeans, s = 50, cmap = 'viridis')

centers = kmeans.cluster_centers_

plt.scatter(centers[:, 0], centers[:, 1], s = 200, alpha = 0.5);
plt.show()

итогом анализа получилось графическое представление выше